In [1]:
import requests
import ipywidgets as widgets
from IPython.display import display
import json
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import time

In [2]:
from code import interact
import pandas as pd
import markdown
from requests import options 
# Create a widget that allows you to flip through all of the events, 
# and displays the event data and coordinates on the ice rink image.
def get_game_ids(path):
    """returns a list of all game ids inside a json file"""
    game_ids = []
    with open(path) as json_file:
        data = json.load(json_file)
        game_ids = list(data.keys())
    return game_ids

def ID_Widget(path):
    """"returns Game ID widget """
    MIN_ID = min(get_game_ids(path))
    MAX_ID = max(get_game_ids(path))
    id_widget = widgets.SelectionSlider(
        value=MIN_ID,
        min=MIN_ID,
        max=MAX_ID,
        # step=1,
        options = get_game_ids(path),
        description='game_id:',
        continuous_update=False
    )
    return id_widget

def display_details(game_id, path):
    """displays the event start date and time according to selected game_id"""
    with open(path) as json_file:
        data = json.load(json_file)
        game = data[str(game_id)]
        a = "Game Start Date: " + game['gameData']['datetime']['dateTime']
        b = "Game ID: " + str(game_id)+"; "+game['gameData']['teams']['home']['abbreviation']+" (home) vs "+game['gameData']['teams']['away']['abbreviation']+" (away)"
        # Display the 2 teams information in a form of a dataframe
        df = pd.DataFrame({'Team': [game['gameData']['teams']['home']['abbreviation'], game['gameData']['teams']['away']['abbreviation']],
                            'Goals': [game['liveData']['linescore']['teams']['home']['goals'], game['liveData']['linescore']['teams']['away']['goals']],
                            'SOG': [game['liveData']['linescore']['teams']['home']['shotsOnGoal'], game['liveData']['linescore']['teams']['away']['shotsOnGoal']],
                            'SO Goals': [game['liveData']['linescore']['teams']['home']['powerPlay'], game['liveData']['linescore']['teams']['away']['powerPlay']],
                            #'SO Attempts': [game['liveData']['linescore']['teams']['home']['powerPlayOpportunities'], game['liveData']['linescore']['teams']['away']['powerPlayOpportunities']]
                            })
        display(a)
        display(b)
        display(df)

'Game Start Date: 2017-04-12T23:00:00Z'

'Game ID: 2016030111; MTL (home) vs NYR (away)'

,Team,Goals,SOG,SO Goals
0,MTL,0,31,False
1,NYR,2,31,False


In [3]:
widgets.interact(display_details, game_id=ID_Widget('./2016_playoffs.json'), path=widgets.fixed('./2016_playoffs.json'))

interactive(children=(SelectionSlider(continuous_update=False, description='game_id:', options=('2016030111', …

<function __main__.display_details(game_id, path)>

In [7]:
def get_event_ids(game_id, path):
    """returns a list of all event ids inside a json file"""
    event_ids = []
    with open(path) as json_file:
        data = json.load(json_file)
        game = data[str(game_id)]
        #event_ids = list(game['liveData']['plays']['allPlays'].keys())
        for event in game['liveData']['plays']['allPlays']:
            event_ids.append(event['about']['eventIdx'])
    return event_ids

def Event_Widget(game_id, path):
    """"returns Event ID widget """
    MIN_EVENT = min(get_event_ids(game_id, path))
    MAX_EVENT = max(get_event_ids(game_id, path))
    event_widget = widgets.IntSlider(
        value=MIN_EVENT,
        min=MIN_EVENT,
        max=MAX_EVENT,
        options = get_event_ids(game_id, path),
        description='event_id:',
        continuous_update=False
    )
    return event_widget

def event_change(event_id, game_id, path):
    """displays event details according to selected game_id and event_id"""
    with open(path) as json_file:
        data = json.load(json_file)
        game = data[str(game_id)]
        event = game['liveData']['plays']['allPlays'][event_id]
        # Display the event details in a form of a dataframe
        df = pd.DataFrame({'Event ID': [event_id],
                            'Event Type': [event['result']['eventTypeId']],
                            'Event Description': [event['result']['description']],
                            'Event Team': [event['team']['name']],
                            'Event Period': [event['about']['period']],
                            'Event Period Type': [event['about']['periodType']],
                            'Event Time': [event['about']['periodTime']],
                            'Event Time Remaining': [event['about']['periodTimeRemaining']],
                            'Event Strength': [event['result']['strength']['name']],
                            'X': [event['coordinates']['x']],
                            'Y': [event['coordinates']['y']]
                            })
        display(df)